# Análisis de sentimiento

¿Recuerdas qué es el NLP (Procesamiento de Lengauje Natural)? Hace tiempo, cuando estudiamos *Feature Selection*, vimos que no podíamos alimentar unmodelo directamente con datos que no fueran numéricos, por lo que si teníamos textos había que ingeniárselas para traducir esa información de modo que "la entienda" nuestro modelo.

A continuación, vamos a repasar lo que vimos en su día y profundizaremos en su aplicación final, donde utilizaremos alguno de los modelos que ya controlamos. Con esto, podrás enlazar los puntos que quedaban sueltos.

Centrándonos en el problema que vamos a utilizar de referencia, partiremos de dos documentos de texto con opiniones de películas en IMDB. Tendremos un documento con frases positivas y otro con frases negativas. Evidentemente, necesitaremos esta clasificación para que nuestro modelo pueda aprender. En este caso, para facilitarnos las cosas, el ratio de frases positivas y negativas será aproximadamente el mismo.

### Importamos librerías

In [2]:
import numpy as np
import pandas as pd
import os
import re
import warnings
warnings.filterwarnings("ignore")

### Cargando los datos

A diferencia de otros ejemplos que hemos visto con anterioridad, no tendremos un dataset como tal, sino que tendremos unos ficheros de texto con frases que deberemos tratar. Además, ya que lo tenemos así, utilizaremos esta estructura de datos (listas) para realizar el análisis que, dado que es un iterable, nos servirá del mismo modo que si fuera un Series.

In [3]:
reviews_train_positivo = []
for line in open('data/train_pos.txt', 'r', encoding='latin1'):
    reviews_train_positivo.append(line.strip())
    
reviews_train_negativo = []
for line in open('data/train_neg.txt', 'r', encoding='latin1'):
    reviews_train_negativo.append(line.strip())
    
reviews_test = []
for line in open('data/test.txt', 'r', encoding='latin1'):
    reviews_test.append(line.strip())

Juntamos las reviews de train:

In [4]:
reviews_train = reviews_train_positivo + reviews_train_negativo

Nos creamos la lista del ``target`` en base a los datos que hemos leído:

In [5]:
target_train = [1 if i < len(reviews_train_positivo) else 0 for i in range(len(reviews_train_positivo) + len(reviews_train_negativo))]

# Y los datos de test, que serán 50/50:
target_test = [1 if i < len(reviews_test)/2 else 0 for i in range(len(reviews_test))]

# Y nos creamos el target como unión de los dos, ya que estarán en orden:
target = target_train + target_test

Y nos aseguramos leyendo un par de frases:

In [6]:
reviews_train[4]

'This is not the typical Mel Brooks film. It was much less slapstick than most of his movies and actually had a plot that was followable. Leslie Ann Warren made the movie, she is such a fantastic, under-rated actress. There were some moments that could have been fleshed out a bit more, and some scenes that could probably have been cut to make the room to do so, but all in all, this is worth the price to rent and see it. The acting was good overall, Brooks himself did a good job without his characteristic speaking to directly to the audience. Again, Warren was the best actor in the movie, but "Fume" and "Sailor" both played their parts well.'

In [7]:
reviews_train[20001]

"I thought that My Favorite Martian was very boring and drawn out!! It was not funny at all. The audience just sat through the whole movie and didn't laugh at all!!! Not even the kids laughed!! That is sad for a Disney movie!! I thought they could have found somebody better to play the martian rather than Christopher Lloyd!! He was really stupid!! And he was not funny!! I thought the talking suit was really dumb!!! In the original television series the suit doesn't talk and move around!! In my opinion they should not have wasted their time on this movie!! I give it two thumbes down!! Really a waste of time and I would not recommend the movie to anybody!!! Thank You!!"

Como puedes observar, la primera opinión es de las buenas y la segunda de las malas.


### Pretratamiento de textos


Por otra parte, si queremos analizar estas frases, no podemos utilizarlas directamente (bueno sí que podríamos pero no sería nuestra mejor opción). Si queremos sacar el máximo partido a nuestros datos de texto, tendremos que hacer un pretratamiento de datos, como vimos hace tiempo.

¿Y este tratamiento en qué consiste? Pues si recuerdas bien, el primer paso consistía en limpiar todo aquello que no sea puro texto, como pueden ser caracteres de puntuación o marcas de texto (estamos ante una extracción HTML).

Por lo tanto, vamos a eliminar todos estos caracteres raros que no aportan información, lo cual podremos hacer de diferentes modos. En este caso, lo que haremos será eliminar los signos de puntuación (como ``,``, ``.``, ``?``...) y sustituir las etiquetas HTML y los caracteres separadores de palabras (como ``-`` o ``/``) por espacios. Además, hemos visto que la distinción entre mayúsculas y minúsculas también nos puede jugar malas pasadas, por lo que lo convertiremos todo a minúsculas:

In [8]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

Veamos el antes y el después de la frase positiva que habíamos leído:

**Antes**:

In [9]:
reviews_train[4]

'This is not the typical Mel Brooks film. It was much less slapstick than most of his movies and actually had a plot that was followable. Leslie Ann Warren made the movie, she is such a fantastic, under-rated actress. There were some moments that could have been fleshed out a bit more, and some scenes that could probably have been cut to make the room to do so, but all in all, this is worth the price to rent and see it. The acting was good overall, Brooks himself did a good job without his characteristic speaking to directly to the audience. Again, Warren was the best actor in the movie, but "Fume" and "Sailor" both played their parts well.'

**Después**:

In [10]:
reviews_train_clean[4]

'this is not the typical mel brooks film it was much less slapstick than most of his movies and actually had a plot that was followable leslie ann warren made the movie she is such a fantastic under rated actress there were some moments that could have been fleshed out a bit more and some scenes that could probably have been cut to make the room to do so but all in all this is worth the price to rent and see it the acting was good overall brooks himself did a good job without his characteristic speaking to directly to the audience again warren was the best actor in the movie but fume and sailor both played their parts well'

## Vectorización

Bien, ya hemos conseguido extraer la información de la que podremos extraer patrones para poder crear nuestro modelo. Sin embargo, nuestro modleo no sabe cómo tratar estos datos de manera directa, no entiende los ``strings``. Para que entienda la información, hay que convertirla a datos numéricos, a lo que hemos llamado *vectorización*.

La forma más sencilla de hacer esto es crear una matriz gigante donde llevemos el recuento de cada una de las palabras que aparecen en todos los textos, aplicado a cada frase. Es decir, crearnos una columna por cada palabra única en la suma de todas y cada una de las frases de nuestro conjunto de frases de entrenamiento, y rellenar cada registro (cada frase) haciendo las cuentas por frase.

Para esto podemos utilizar el método ``CountVectorizer``, que nos dejará la opción de hacer el conteo o simplemente registrar si tiene esa palabra (1) o no (0), lo cual podemos establecer con el parámetro ``binary``.

*NOTA*: Este método nos devolverá una matriz *sparse*, que si bien recuerdas era una matriz que tenía una forma especial para ocupar menos espacio en memoria, pero que para tratarlas había que convertirlas con ``toarray()``:

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

baseline_vectorizer = CountVectorizer(binary=False)
baseline_vectorizer.fit(reviews_train_clean)

X_baseline = baseline_vectorizer.transform(reviews_train_clean)
X_test_baseline = baseline_vectorizer.transform(reviews_test_clean)

In [12]:
print(X_baseline.shape)
# baseline_vectorizer.vocabulary_ # ordenado alfabéticamente

(25000, 87063)


In [13]:
# # Para obtener solo las palabras o nombres de las columnas que vamos a utilizar:
# baseline_vectorizer.get_feature_names()

# Entrenando el modelo

Utilizaremos un modelo de regresión lineal para crer nuestro detector de sarcasmo, a ver qué tal se nos da. En este caso, parece que es una buena idea, ya que los modelos lineales:
* Son fáciles de interpretar (podremos sacar fácilmente la importancia de cada n_grama
* Debido a su naturaleza tan "simple" puede aprender mucho más rápido que otros algoritmos
* Trabajan bien con matrices sparse

Para probar el modelo, haremos un poco de GridSearchCV, que es algo que nos encanta. En este caso, como parámetros a modificar no tendremos muchas opciones, por lo que solamente variaremos ``C``, probando [0.01, 0.05, 0.25, 0.5, 1] para encontrar el valor que mejor accuracy nos devuelva:

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline


def train_model(X_TRAIN, X_TEST, TARGET_TRAIN, TARGET_TEST, model=LogisticRegression()):
    
    # Creamos pipeline:
    pipe = Pipeline([('model', model)])
    
    # Creamos parámetros:
    params = {
        'model__C': [0.01, 0.05, 0.25, 0.5, 1]
    }
    
    # Creamos grid:
    grid = GridSearchCV(pipe,
                        params,
                        cv=5,
                        n_jobs=-1,
                        scoring='accuracy',
                        verbose=4
                       )
    
    # Entrenamos:
    grid.fit(X_TRAIN, TARGET_TRAIN)
    
    # Y predecimos:
    print (f"Final Accuracy: {accuracy_score(TARGET_TEST, grid.best_estimator_.predict(X_TEST))}")
    print (f"Best params: {grid.best_params_}")
    return grid
    
train_model(X_baseline, X_test_baseline, target_train, target_test)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Final Accuracy: 0.88072
Best params: {'model__C': 0.01}


GridSearchCV(cv=5, estimator=Pipeline(steps=[('model', LogisticRegression())]),
             n_jobs=-1, param_grid={'model__C': [0.01, 0.05, 0.25, 0.5, 1]},
             scoring='accuracy', verbose=4)

## Eliminando Stop Words

¿Qué eran las *stop words*? Si no lo recuerdas, las *stop words* eran aquellas palabras tan comunes que aparecen tantas veces que no nos aportan información, como ``if``, ``but``, ``we``, ``he`` o ``she``. Normalmente, eliminar estas palabras no cambia el sentido semántico de un texto y SUELE dar mejores resultados. Esto será mucho más útil cuanto mayores sean las secuencias de palabras (n-grams).

Así que, ahora, vamos a aplicar el ``CountVectorizer`` tras eliminar las *stop words* que vienen por defecto en el paquete ``nltk``, que son las más comunes, aunque podríamos utilizar otras propias.

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TheBridge\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
from nltk.corpus import stopwords
stopwords.words('english')[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [19]:
from nltk.corpus import stopwords

english_stop_words = stopwords.words('english')
def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        
        # Para cada review elimina las stopwords, y separa todas las palabras por espacio
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

no_stop_words_train = remove_stop_words(reviews_train_clean)
no_stop_words_test = remove_stop_words(reviews_test_clean)

In [20]:
cv = CountVectorizer(binary=True)
cv.fit(no_stop_words_train)

X = cv.transform(no_stop_words_train)
X_test = cv.transform(no_stop_words_test)

In [21]:
print(X_baseline.shape)
print(X.shape)
print("Stop words eliminadas:", X_baseline.shape[1] - X.shape[1])

(25000, 87063)
(25000, 87046)
Stop words eliminadas: 17


In [22]:
cv = CountVectorizer(binary=True,
                    stop_words=english_stop_words)

cv.fit(reviews_train_clean)

X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

train_model(X, X_test, target_train, target_test)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Final Accuracy: 0.87976
Best params: {'model__C': 0.05}


GridSearchCV(cv=5, estimator=Pipeline(steps=[('model', LogisticRegression())]),
             n_jobs=-1, param_grid={'model__C': [0.01, 0.05, 0.25, 0.5, 1]},
             scoring='accuracy', verbose=4)

In [23]:
print(X_baseline.shape)
print(X.shape)
print("Stop words eliminadas:", X_baseline.shape[1] - X.shape[1])

(25000, 87063)
(25000, 86918)
Stop words eliminadas: 145


*NOTA:* Como hemos mencionado, eliminar las *stop words* SUELE (no siempre) mejorar el modelo. En este caso, como puedes comprobar, resulta que nos reduce el *performance* del modelo, por lo que no será buena idea utilizarlo en este en concreto.

Si te has fijado, hemos utilizado el parámetro ``stop_words`` para elegir la lista de *stop words* que queremos eliminar. En este caso ha sido la lista por defecto, pero podríamos utilizar otra que hayamos analizado que sea mejor, como podría ser ['in', 'of', 'at', 'a', 'the'] .

Otro paso típico que se suele utilizar en este tipo de problemas consiste en normalizar las palabras del corpus mediante la extracción del significado base de cada una de ellas. Para esto, utilizaremos dos técnicas: *Stemming* y *Lematización*.

## Stemming

El Stemming es considerado una técnica de normalización por aproximación de fuerza bruta, aunque no por ello significa que sea peor. Hay varios algoritmos, pero por lo general se usarán reglas básicas para cortar las terminaciones de las palabras.

NLTK tiene varias implmentaciones de los algoritmos de *Stemming*:
* PorterStemmer
* SnowballStemmer

A continuación podemos ver un ejemplo de la implementación de cada uno de ellos:

In [24]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

plurals = ['caresses', 'flies', 'dies', 'mules', 'denied',
            'died', 'agreed', 'owned', 'humbled', 'sized',
            'meeting', 'stating', 'siezing', 'itemization',
            'sensational', 'traditional', 'reference', 'colonizer',
            'plotted']

singles = [stemmer.stem(i) for i in plurals]

print(' '.join(singles))

caress fli die mule deni die agre own humbl size meet state siez item sensat tradit refer colon plot


In [26]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

plurals = ['caresses', 'flies', 'dies', 'mules', 'denied',
            'died', 'agreed', 'owned', 'humbled', 'sized',
            'meeting', 'stating', 'siezing', 'itemization',
            'sensational', 'traditional', 'reference', 'colonizer',
            'plotted']
singles = [stemmer.stem(plural) for plural in plurals]

print(' '.join(singles))

caress fli die mule deni die agre own humbl size meet state siez item sensat tradit refer colon plot


Como puedes observar, los resultadso son muy parecidos, y es que ambos algoritmos son similares, donde podemos ver SnowballStemmer como una versión mejorada, un poco más agresiva que el PorterStemmer (más clásico y ligeramente más conservador).

En este caso, utilizaremos el PorterStemmer por ser más conservador: 

In [ ]:
def get_stemmed_text(corpus):
    from nltk.stem.porter import PorterStemmer
    stemmer = PorterStemmer()

    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

stemmed_reviews_train = get_stemmed_text(reviews_train_clean)
stemmed_reviews_test = get_stemmed_text(reviews_test_clean)

cv = CountVectorizer(binary=True)
cv.fit(stemmed_reviews_train)
X_stem = cv.transform(stemmed_reviews_train)
X_test = cv.transform(stemmed_reviews_test)

train_model(X_stem, X_test, target_train, target_test)

In [ ]:
print(X_baseline.shape)
print(X_stem.shape)
print("Diff X normal y X tras stemmer y vectorización:", X_baseline.shape[1] - X_stem.shape[1])

## Lematización

La técnica de *Lematización* trabaja identificando la parte gramatical de una palabra dada, aplicando tras ello reglas más complejas para transformar la palabra en su verdadera raíz.

Veamos un ejemplo:

In [ ]:
# Para castellano deberíamos instalar la siguiente librería:
# pip install es-lemmatizer   para castellano

# Para inglés:
nltk.download('wordnet')

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

plurals = ['caresses', 'flies', 'dies', 'mules', 'studies',
            'died', 'agreed', 'owned', 'humbled', 'sized',
            'meeting', 'stating', 'siezing', 'itemization',
            'sensational', 'traditional', 'reference', 'colonizer',
            'plotted']
singles = [lemmatizer.lemmatize(plural) for plural in plurals]

print(' '.join(singles))

Si bien antes no observábamos apenas diferencia entre PorterStemmer y SnowballStemmer, ahora sí que se observan resultados diferentes. No hay un método mucho mejor que otro, simplemente son diferentes, por lo que no podemos decir que siempre va a ser mejor utilizar Lematización frente a Stemmer, o viceversa.

In [ ]:
def get_lemmatized_text(corpus):
    
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]

# Lematizamos las reviews
lemmatized_reviews_train = get_lemmatized_text(reviews_train_clean)
lemmatized_reviews_test = get_lemmatized_text(reviews_test_clean)

# Vectorizamos con conteo tras lematizar
cv = CountVectorizer(binary=True)
cv.fit(lemmatized_reviews_train)

X = cv.transform(lemmatized_reviews_train)
X_test = cv.transform(lemmatized_reviews_test)

train_model(X, X_test, target_train, target_test)

In [ ]:
print(X_baseline.shape)
print(X.shape)
print("Diff X normal y X tras lematizador y vectorización:", X_baseline.shape[1] - X.shape[1])

## n-gramas

Existe una manera mediante la cual podemos agregar más poder predictivo a nuestro modelo: agregar secuencias de palabras, generalmente de 2 o 3 palabras (bigramas o trigramas).

Por ejemplo, si una reseña tuviera la secuencia de 3 palabras ``didn't love movie``, solo consideraríamos estas palabras individualmente con un modelo unigrama, dejando sin capturar el significado real de la frase, pues no enlazaríamos ``didn't`` con ``love``, que tiene una connotación negativa, sino que estaríamos viendo la palabra ``love`` por ahí, lo que tiene connotaciones positivas.

Gracias a scikit-learn esto se vuelve realmente fácil (como todo lo que llevamos visto hasta ahora). Solamente utilizando el argumento ``ngram_range`` con cualquiera de las clases ``Vectorizer`` nos valdrá:


In [ ]:
from nltk import ngrams

sentence = 'this is foo bar'
sentence2 = 'this'
sentence3 = 'foo bar'

two = ngrams(sentence.split(),2)
three = ngrams(sentence.split(),3)

for grams in two:
    print(grams)

print('###############')
for grams in three:
    print(grams)

ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1,3))

vector = ngram_vectorizer.fit_transform([sentence,sentence2, sentence3]).toarray()
print(vector)
print(len(vector[0]))

In [ ]:
# Podremos extaer el vocabulario (como antes):
ngram_vectorizer.vocabulary_

Vamos a probarlo sobre nuestro modelo:

In [ ]:
ngram_vectorizer = CountVectorizer(binary=True,
                                  ngram_range=(1,2))

ngram_vectorizer.fit(reviews_train_clean)

X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

train_model(X, X_test, target_train, target_test)

In [ ]:
print(X_baseline.shape)
print(X.shape)
print("Diff X normal y X tras lematizador y vectorización:", X_baseline.shape[1] - X.shape[1])

Vaya, parece que ahora sí que hemos mejorado el *performance* del modelo, ¡¡y con menos variables!!

## TF-IDF

Otra técnica muy extendida que también habíamos visto en el pasado era la técnica TF-IDF (Term Frequency-Inverse Document Frequency), que consiste en un estadístico para cada palabra al que se le asigna un factor de ponderación que podemos usar en lugar de representaciones binarias o de conteo de palabras.

Hay varias formas de realizar la transformación TF-IDF, pero, resumiendo,  se trata de una técnica que tiene como objetivo representar la cantidad de veces que una palabra determinada aparece en un documento (una opinión de una película, en nuestro caso) en relación a la cantidad de documentos en el corpus en el que aparece la palabra, donde las palabras que aparecen en muchos documentos tienen un valor más cercano a cero y las palabras que aparecen en menos documentos tienen valores más cercanos a 1.

Veamos un ejemplo:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# ln(N+1/df+1) +1

sent1 = 'My name is Ralph'
sent2 = 'Ralph is fat'
sent3 = 'Ralph'

test = TfidfVectorizer()
test.fit_transform([sent1, sent2, sent3])
print(test.idf_)
print(test.get_feature_names())

Probemos ahora sobre nuestro dataset:

In [ ]:
tfidf_vectorizer = TfidfVectorizer()

tfidf_vectorizer.fit(reviews_train_clean)

X = tfidf_vectorizer.transform(reviews_train_clean)
X_test = tfidf_vectorizer.transform(reviews_test_clean)


train_model(X, X_test, target_train, target_test)

## Support Vector Machines (SVM)

Hasta ahora, hemos probado estas técnicas con un modelo LogisticRegression, que es un modelo lineal. Por lo general, este tipo de problemas con *sparse datasets*, es decir, con conjuntos de datos con muchos 0 (como lo que tenemos nosotros,), suelen responder mejor ante modelos lineales, ya que podemos particularizr de mejor manea el efecto de cada palabra, además de entrenar relativamente más rápido que con otros modelos más complejos.

Por lo tanto, si estamos hablando de algoritmos lineales, podríamos probar otro algoritmo lineal, como es el SVC con kernel lineal:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# SVM con bigramas:
ngram_vectorizer = CountVectorizer(binary=True,
                                   ngram_range=(1, 2))

ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

train_model(X, X_test, target_train, target_test, model=LinearSVC())

## Versión final

Tras diferentes iteraciones, se ha llegado a que los mejores resultados se obtienen con la eliminación de un pequeño conjunto de *stop words*, junto con un rango de n-gramas de 1 a 3 y un SVC lineal, tal como se muestra a continuación:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC


stop_words = ['in', 'of', 'at', 'a', 'the']

ngram_vectorizer = CountVectorizer(binary=True,
                                   ngram_range=(1, 3),
                                   stop_words = stop_words)

ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)


train_model(X, X_test, target_train, target_test, model=LinearSVC())

## Top Palabras: positivas vs. negativas

Con el modelo lineal, podemos obtener la importancia de cada variable en base al valor de sus coeficientes:

In [ ]:
cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)

log_reg = LogisticRegression(C=0.5)
log_reg.fit(X, target)

#print(log_reg.coef_)
#print(cv.get_feature_names())

# Montamos un diccionario con palabra -> coeficiente
feature_to_coef = {
    word: coef for word, coef in zip(cv.get_feature_names(), log_reg.coef_[0])
}

In [ ]:
print("TOP Positivas:")

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:5]:
    print(best_positive)
    
print('---------------------------')
print("TOP Negativas:")

for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:5]:
    print(best_negative)